This code generates the model SNe for all of the models. Check up on Model 4, I think the mixed map has a normalization error. (map built in 3.4cModelVisualization)

In [1]:
#import packages and initialize settings
import time
import os
import numpy as np
from astropy.io import fits, ascii
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table


plt.rc('text',usetex=True)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

import sys
# Work Laptop:     
#sys.path.append('/Users/mayker.1/Desktop/NessPythonFunctions')
# Work Desktop:    
sys.path.append('/home/mayker.1/Desktop/PythonFunctions')
# Personal Laptop: sys.path.append('/Users/nessmayker/Desktop/NessPythonFunctions')
from FindNearestMC import nearestMCMethod
from SNeGenerationModels import runModels

In [2]:
#Get Galaxy names, distances, and fits file locations

# Unpack master table to get SNe information
database = Table.read('../Data/3.SNe+GalData.csv', format = "csv")
centerCoords = [(database["orient_ra"][i], database["orient_dec"][i]) for i in range(len(database))]   

database[0:1]



galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,telOrient,ImageFile150,ErrFile150,AlphaCOFile,PACovFile,IFRFile,MixedMaps,IFRResizedMaps,MassMaps
str8,str11,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str112,str113,str74,str112,str97,str37,str54,str35
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078,213.29124,-65.33915,4.2,36.74,64.3,262.49515,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/ESO097-013_alphaCO21_Sun+20.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_coverage2d.fits,/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/z0mgs/eso097-013_w1_gauss7p5.fits,../Data/MixedMaps/circinus_Mixed.fits,../Data/ReprojectedInfraredMaps/circinus_reshaped.fits,../Data/MassMaps/circinus_Mass.fits


In [3]:

#Make lists unique for galaxies (no repeating galaxies or other data)    
def unique(origGalsList, origDistList, imageFileList, errFileList, centerCoords, posAngList, inclList, covFileList, ifrFileList, massFileList, mixedFileList): 
    gals, dists, images, errFiles, coords, posAngs, inclinations, covFiles, ifrFiles, massFiles, mixedFiles = [],[],[],[],[],[],[],[],[],[],[]
    for i in range(len(origGalsList)): 
        if origGalsList[i] not in gals: 
            if origGalsList[i] != "ngc1068" and origGalsList[i] != "ngc1672" and origGalsList[i] != "ngc4579":
                gals.append(origGalsList[i]) 
                dists.append(origDistList[i])
                images.append(imageFileList[i])
                errFiles.append(errFileList[i])
                coords.append(centerCoords[i])
                posAngs.append(posAngList[i])
                if inclList[i] == 90.0:
                    inclList[i] = 80.0
                inclinations.append(inclList[i]) 
                covFiles.append(covFileList[i])
                ifrFiles.append(ifrFileList[i])
                massFiles.append(massFileList[i])
                mixedFiles.append(mixedFileList[i])
            else:
                pass
    return(gals, dists, images, errFiles, coords, posAngs, inclinations, covFiles, ifrFiles, massFiles, mixedFiles)

gals, dists, images, errFiles, coords, posAngs, inclinations, covFiles, ifrFiles, massFiles, mixedFiles = unique(database["galaxy"], database["dist"],database["ImageFile150"], database["ErrFile150"], centerCoords, database["orient_posang"], database["orient_incl"], database["PACovFile"], database["IFRResizedMaps"], database["MassMaps"], database["MixedMaps"])
print(gals)


['circinus', 'ngc0253', 'ngc0300', 'ngc0628', 'ngc1087', 'ngc1097', 'ngc1365', 'ngc1433', 'ngc1511', 'ngc1559', 'ngc1566', 'ngc1637', 'ngc2775', 'ngc2997', 'ngc3239', 'ngc3627', 'ngc4254', 'ngc4303', 'ngc4321', 'ngc4424', 'ngc4457', 'ngc4496a', 'ngc4536', 'ngc4945', 'ngc5128', 'ngc5236', 'ngc5530', 'ngc6744', 'ngc7793']


In [4]:
    
# run models 1-4 and store results
galCol, counter, n55mod1, n65mod1, n55mod2, n65mod2, n55mod3, n65mod3, n55mod4, n65mod4 = [],[],[],[],[],[],[],[],[],[]

for i in range(len(gals)):
    
    ifr_hdu = pyfits.open(ifrFiles[i])
    ifr_map = ifr_hdu[0].data
    cov_hdu = pyfits.open(covFiles[i])
    cov_map = cov_hdu[0].data

    wcs     = WCS(cov_hdu[0].header, naxis=2)
    naxis   = wcs._naxis # size of image naxis[0] = x and [1] = y
    grid    = np.indices((naxis[1],naxis[0]))
    ra, dec = wcs.wcs_pix2world(grid[1],grid[0],0) 

    f_cov = cov_map.flatten()
    f_ifr = ifr_map.flatten() 
    f_ra  = ra.flatten()
    f_dec = dec.flatten()

    keep = np.where((f_cov > 0.9) & np.isfinite(f_ifr))
    ifr  = f_ifr[keep]
    star_ra   = f_ra[keep]
    star_dec  = f_dec[keep]
    
    
    raArr1, decArr1, dxArr1, dyArr1 = runModels(gals[i], images[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=1)
    raArr2, decArr2, dxArr2, dyArr2 = runModels(gals[i], images[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=2)
    raArr3, decArr3, dxArr3, dyArr3 = runModels(gals[i], images[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=3, starLight = ifr, starRa = star_ra, starDec = star_dec)
    raArr4, decArr4, dxArr4, dyArr4 = runModels(gals[i], mixedFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=2)

    n55_mod1, n65_mod1 = nearestMCMethod(gals[i], images[i], errFiles[i], massFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr1, decArr1)
    n55_mod2, n65_mod2 = nearestMCMethod(gals[i], images[i], errFiles[i], massFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr2, decArr2)
    n55_mod3, n65_mod3 = nearestMCMethod(gals[i], images[i], errFiles[i], massFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr3, decArr3)
    n55_mod4, n65_mod4 = nearestMCMethod(gals[i], images[i], errFiles[i], massFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr4, decArr4)
    
    if (type(n55_mod1) != float and type(n55_mod1) != np.float64):
        for j in range(len(n55_mod1)):
            galCol.append(gals[i])
            counter.append(j)
            n55mod1.append(n55_mod1[j])
            n55mod2.append(n55_mod2[j])
            n55mod3.append(n55_mod3[j])
            n55mod4.append(n55_mod4[j])
    else:
        for j in range(100):
            galCol.append(gals[i])
            counter.append(j)
            n55mod1.append(np.float('nan'))
            n55mod2.append(np.float('nan'))
            n55mod3.append(np.float('nan'))
            n55mod4.append(np.float('nan'))           
        
    if (type(n65_mod1) != float and type(n65_mod1) != np.float64):
        for j in range(len(n65_mod1)):
            n65mod1.append(n65_mod1[j])
            n65mod2.append(n65_mod2[j])
            n65mod3.append(n65_mod3[j])
            n65mod4.append(n65_mod4[j])
    else:
        for j in range(100):
            n65mod1.append(np.float('nan'))
            n65mod2.append(np.float('nan'))
            n65mod3.append(np.float('nan'))
            n65mod4.append(np.float('nan'))      
    
results = Table({"Galaxy": galCol,
                "ModelNum": counter,
                "n55Mod1": n55mod1,
                "n65Mod1": n65mod1,
                "n55Mod2": n55mod2,
                "n65Mod2": n65mod2,
                "n55Mod3": n55mod3,
                "n65Mod3": n65mod3,
                "n55Mod4": n55mod4,
                "n65Mod4": n65mod4})

results.write("../Data/AllModels.csv", format = "csv", overwrite = True)

        
        
    

at Mass cutoff A for circinus
Nearest 55 [ 363.45358371  322.83704549  194.23874223  242.72503733  242.5302979
  242.51411741  254.63162498   60.02649615  723.63810547    0.
    0.            0.            0.         1879.23924746    0.
 1028.81725155    0.            0.          908.12406157 1043.00362532
    0.            0.          443.22765293  236.95470785   60.13718521
  581.51979906  765.72527583  646.83895402    0.           97.09804375
  319.87826216    0.          236.2758263   646.15971369    0.
  124.52821063 1189.41863171    0.          822.25149448   60.15635375
  322.95292354  337.83812785  555.02122404  930.02878111  147.76639591
    0.            0.            0.          440.79301328   60.13084432
    0.          129.89139946    0.            0.          261.22955718
   60.1243292     0.            0.          471.08690757    0.
    0.            0.          178.44131287    0.            0.
  129.84671844  521.5787948   379.32409393    0.          892.7433985
  191.9

at Mass cutoff A for circinus
Nearest 55 [2252.78514818 3745.14134162 2124.57523115  191.99622129    0.
  498.2371579     0.            0.          661.66008657    0.
    0.         6313.5227723    60.02105665    0.            0.
    0.          180.22679518    0.         1131.78531034    0.
    0.            0.            0.            0.            0.
 1875.25509059    0.          120.13271964    0.          235.48192501
 3625.90217607    0.            0.          203.56778758 2775.97944892
  531.44957426    0.          956.58107519    0.          407.45420141
    0.         1387.94268121    0.            0.            0.
  120.07155168    0.          191.98627579 1021.56313627    0.
    0.          961.91151195    0.         2617.28442606    0.
  876.60375368   60.11517535    0.            0.            0.
    0.            0.          939.25516315  129.92941487    0.
   60.06614831  963.73225529 1851.23222329    0.            0.
    0.            0.            0.          120.11314

at Mass cutoff A for ngc0253
Nearest 55 [   0.            0.            0.            0.            0.
  179.0394444     0.          668.9918238   267.59543398  306.71320936
    0.            0.            0.          304.71215395    0.
    0.            0.            0.            0.            0.
    0.          707.48665296    0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
  528.68903574    0.            0.            0.          415.90299744
 1086.81013874    0.          832.39375357    0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.          133.22033346
   98.54013229   98.52887862    0.          596.36805515    0.
    0.            0.          519.9864598     0.            0.
   56.41460363    0.            0.          153.70296298    0.
    0.           98.52580641    0.            0.      

at Mass cutoff A for ngc0300
Nearest 55 [ 378.31105672  483.86862973  901.49126288  210.37140611 1324.20037133
  500.23053572  733.75835566 1239.3127461   282.60550106  652.23663474
  118.70187095 1108.5562792   629.29016216  118.68131898   21.41691302
 1011.60591732  304.8300735   913.01386207  929.2708819   315.24619339
 1189.81332764  781.9669506   449.61464177  753.0252668   696.46322367
  625.57255579  370.36880118  834.14240052   64.29431439  251.46944741
  508.7856923   283.73180078  734.27688544  816.05055859  327.04375004
    0.          692.17899101  375.78684133  567.00050175  183.53566579
 1265.40043395  109.18165762  783.04562576  761.50647386 1145.48309941
  548.02592759  221.60594978 1117.78087704  447.99232242  107.88207645
  611.69481903  313.87151511  279.87500579  472.31514006  149.91192362
  263.56620528 1274.23084828  548.00327154  308.86088583 1054.88466298
  126.06808523  802.54815678  544.02473363  199.30952111 1002.53378112
 1743.82188842   36.34131511  755.320

<ipython-input-4-4c21bd7cd7f2>:62: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  n65mod1.append(np.float('nan'))
<ipython-input-4-4c21bd7cd7f2>:63: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  n65mod2.append(np.float('nan'))
<ipython-input-4-4c21bd7cd7f2>:64: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. T

...

at Mass cutoff A for ngc5128
Nearest 55 [ 287.19128044  203.14599862    0.            0.          139.56415236
 1332.17394324    0.            0.            0.            0.
    0.            0.          180.82265206    0.            0.
   40.62224946   81.2681318     0.          469.01949181    0.
  710.96400664    0.           40.63406576 1951.94392053    0.
  927.98524961  162.64996511  337.59059226    0.          968.67415006
   47.1074664    47.11557853  180.69847377    0.            0.
  232.70631937  387.57810144  213.19430066    0.         1518.7317229
    0.            0.          179.49050357   40.6424798     0.
 2824.60001861    0.           81.27681796    0.            0.
    0.            0.          418.69485086    0.         1438.23375203
    0.            0.          792.28922136   80.38294657   94.23138886
    0.            0.            0.            0.           90.39620438
  207.4797665   538.49859105    0.          584.82516003    0.
  139.57763195    0.           

at Mass cutoff A for ngc5236
Nearest 55 [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 12.42591869  0.          0.         28.78057455  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         64.45682964  0.          0.          0.
  0.          0.          0.          0.          0.         64.45281533
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         12.42723116  0.         37.28341104 42.89522098
 73.4170097   0.          0.          0.          0.          0.
  0.          0.          0.      

at Mass cutoff A for ngc5530
Nearest 55 [  0.           0.           0.           0.           0.
   0.         540.99473797  26.44323245   0.           0.
 756.12391943   0.          41.51706709   0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.          26.33212867 189.39382428
   0.         412.09676493   0.           0.           0.
 178.6661949    0.           0.           0.           0.
 560.82279994   0.           0.           0.           0.
 132.21960406   0.          26.32258802   0.         115.92824116
   0.           0.           0.           0.          32.5665216
   0.           0.          26.32569653   0.           0.
   0.           0.         105.12174937 102.72614184   0.
   0.           0.           0.          89.34006033   0.
   0.           0.          41.52485591   0.         128.49220466
 516.51453585   0.         185.07613103   0.           0.
   0.           0.           0.           0.           0.
 

at Mass cutoff A for ngc6744
Nearest 55 [208.82725715 145.42109159 198.36174671 247.2992827    0.
 192.74291287 111.45338111 319.20579142  40.18631833   0.
 274.70283298  43.18671646 167.01931594 423.76068361 276.5468164
   0.         222.49643698   0.         199.30983715 107.14453151
  43.40176127   0.         499.41455043  63.48703713 545.28790027
   0.           0.           0.         137.19676812  23.33373389
   0.          28.65369569 167.04798717  28.66739988 402.35556616
  63.52619996 222.31036819   0.          43.05695661 215.17897712
 174.37471379   0.          14.32782736   0.         437.86830454
 214.99980219   0.          64.54446495   0.          40.18616816
 321.33822935   0.         168.94360984   0.         106.25437201
 341.8452581  608.11799795   0.         236.64355382  69.77554612
   0.           0.           0.          66.39140767   0.
 164.35762538 174.63477604   0.         710.63212892 572.18567358
  14.32418003 122.97862797 306.95052454   0.          63.5639

at Mass cutoff A for ngc6744
Nearest 55 [   0.            0.            0.          654.07503847  184.89870815
 2521.39197603  105.65050946    0.          243.60949291  163.76588933
 1809.25503067  265.54953249 2310.2119598     0.          123.00732472
  224.23402131    0.          261.80965454    0.            0.
  871.34418514    0.           14.32878633   69.76536357    0.
   56.30684231  194.87886144   95.32402986  292.41118849 1989.96497381
    0.          343.85407964   31.75760582   42.99616448    0.
    0.            0.          100.28694593    0.          295.16706783
  153.84364864  185.55235609   64.53171122    0.          148.71766247
  201.26147389  232.98516964 1696.05696513    0.          401.47322327
  122.94303072  530.99887079   57.34141021    0.         1267.49208179
  881.91644698  151.79936379    0.            0.            0.
 1592.18107926 1990.21638664  613.75978751  449.98484692 1940.05659304
   99.69421405  929.09663247    0.            0.          174.1499988